In [32]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook"
colors = px.colors.qualitative.Plotly

In [25]:
# read Schaefer 400 parcellation and construct a dataframe to join with correlation dataframe
# txt file was fetched from https://github.com/ThomasYeoLab/CBIG/blob/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal/Parcellations/HCP/fslr32k/cifti/Schaefer2018_400Parcels_7Networks_order_info.txt
schaefer_nw_df = pd.read_csv("./Schaefer2018_400Parcels_7Networks_order_info.txt", header=None)
schaefer_nw_df = schaefer_nw_df.iloc[::2]
schaefer_nw_df['hemisphere'] = schaefer_nw_df.loc[:, 0].apply(lambda x: x.split('_')[1])
schaefer_nw_df['network'] = schaefer_nw_df.loc[:, 0].apply(lambda x: x.split('_')[2])
schaefer_nw_df['parcel'] = np.arange(400) + 1
schaefer_nw_df.drop(columns=[0], inplace=True)
schaefer_nw_df

,hemisphere,network,parcel
0,LH,Vis,1
2,LH,Vis,2
4,LH,Vis,3
6,LH,Vis,4
8,LH,Vis,5
...,...,...,...
790,RH,Default,396
792,RH,Default,397
794,RH,Default,398
796,RH,Default,399


In [63]:
## Load correlation data between a feature (visual stats or pe) and parcel-average time series

# copied from PE_parcel_Correlations.rnw
# clip names in run.ids order; see analysisPrep.R
run_ids = ["1.2.3", "6.3.9", "3.1.3", "2.4.1"]  # clip names in run.ids order; see analysisPrep.R
# feature = "PECorrs_1_500"
# feature_name = "pe"
feature = "VisualCorrs_1_500"
feature_name = "pixel_change_mean"

df = pd.DataFrame()
for i, r in enumerate(run_ids):
    run_corr = pd.read_table(f'./{feature}/matlabCorrs_{feature_name}_run{i+1}_dtFixed.txt', sep=' ',)
    run_corr['run'] = r
    run_corr['img'] = 'sch' # Schaefer
    df = pd.concat([df, run_corr])

    run_corr = pd.read_table(f'./{feature}/matlabCorrs_{feature_name}_run{i+1}_dtFixed_subcortical.txt', sep=' ',)
    run_corr['run'] = r
    run_corr['img'] = 'subcortical' # Subcortical
    df = pd.concat([df, run_corr])
df.head(5), df.columns

(   sub.id start.at        p1        p2        p3        p4        p5  \
 1  sub-01  corrAt0  0.009397 -0.133796 -0.093123 -0.085996 -0.135573   
 2  sub-02  corrAt0 -0.002728  0.044386 -0.027499 -0.051405 -0.057624   
 3  sub-03  corrAt0  0.179252  0.113261  0.106136  0.137476  0.197730   
 4  sub-04  corrAt0  0.213634  0.193416  0.226295  0.245760  0.243731   
 5  sub-05  corrAt0 -0.032965 -0.145532 -0.032952 -0.026014 -0.006489   
 
          p6        p7        p8  ...      p393      p394      p395      p396  \
 1 -0.124528 -0.111681 -0.004918  ... -0.064563 -0.012540 -0.109964 -0.138157   
 2 -0.031497  0.043078 -0.015555  ... -0.055062 -0.101756 -0.184696 -0.082410   
 3  0.114833 -0.001901  0.082734  ... -0.000339 -0.064484  0.003738  0.004977   
 4  0.231854  0.081805  0.274879  ...  0.021750 -0.000374  0.066337  0.025017   
 5 -0.102065 -0.174684 -0.118382  ... -0.010289  0.085027 -0.160979 -0.086641   
 
        p397      p398      p399      p400    run  img  
 1 -0.110785 -0

In [64]:
# pivot wide to long format
df_no_shift = df[df['start.at'] == "corrAt0"]
df_no_shift = pd.wide_to_long(df_no_shift, stubnames='p', i=['sub.id', 'run', 'img'], j='parcel')
df_no_shift.rename(columns={'p': 'correlation'}, inplace=True)
df_no_shift = df_no_shift.reset_index()
del df # to save RAM memory
df_no_shift.dropna(axis=0, inplace=True)  # dropping rows with parcel ids that are not in the subcortical atlas (the long format has 400 parcels)

In [23]:
df_no_shift

,sub.id,run,img,parcel,start.at,correlation
0,sub-01,1.2.3,sch,1,corrAt0,0.067295
1,sub-01,1.2.3,sch,2,corrAt0,-0.044495
2,sub-01,1.2.3,sch,3,corrAt0,0.216784
3,sub-01,1.2.3,sch,4,corrAt0,0.427122
4,sub-01,1.2.3,sch,5,corrAt0,0.289660
...,...,...,...,...,...,...
150051,sub-47,2.4.1,subcortical,52,corrAt0,0.058396
150052,sub-47,2.4.1,subcortical,53,corrAt0,-0.217750
150053,sub-47,2.4.1,subcortical,54,corrAt0,-0.162983
150057,sub-47,2.4.1,subcortical,58,corrAt0,0.184351


In [65]:
# join with schaefer_nw_df to get network and hemisphere information
df_no_shift = pd.merge(df_no_shift, schaefer_nw_df, on='parcel', how='left')
df_no_shift.loc[df_no_shift.img == "subcortical", "hemisphere"] = None
df_no_shift.loc[df_no_shift.img == "subcortical", "network"] = None

In [66]:
df_no_shift

,sub.id,run,img,parcel,start.at,correlation,hemisphere,network
0,sub-01,1.2.3,sch,1,corrAt0,0.009397,LH,Vis
1,sub-01,1.2.3,sch,2,corrAt0,-0.133796,LH,Vis
2,sub-01,1.2.3,sch,3,corrAt0,-0.093123,LH,Vis
3,sub-01,1.2.3,sch,4,corrAt0,-0.085996,LH,Vis
4,sub-01,1.2.3,sch,5,corrAt0,-0.135573,LH,Vis
...,...,...,...,...,...,...,...,...
78767,sub-47,2.4.1,subcortical,52,corrAt0,-0.003939,None,None
78768,sub-47,2.4.1,subcortical,53,corrAt0,-0.028326,None,None
78769,sub-47,2.4.1,subcortical,54,corrAt0,-0.031384,None,None
78770,sub-47,2.4.1,subcortical,58,corrAt0,0.094765,None,None


In [93]:
# a scatter plot, x is each subject, y is the feature-parcel correlation, facet by run, color by type of atlas (schaefer or subcortical) 
df_select = df_no_shift[df_no_shift['network'].isin(['DorsAttn', 'Vis'])].dropna(axis=0)
# df_select = df_no_shift.dropna(axis=0)
# fig = px.strip(df_select, x='sub.id', y='correlation', color='network', facet_col='run', facet_col_wrap=1)
fig = px.scatter(df_select, x='sub.id', y='correlation', color='network', facet_col='run', facet_col_wrap=1, opacity=0.5)
fig.update_layout(title=f'Correlation between {feature_name} and parcels, {feature}', xaxis_title='Subject', yaxis_title='Correlation')
fig.update_yaxes(range=[-0.5, 0.5])
fig.update_layout(width=800, height=1200)
fig.show()

In [84]:
# a violin plot, x is the run, y is the correlation, facet row by network type, left side is left hemisphere, right side is right hemisphere
fig = make_subplots(rows=4, cols=2, shared_xaxes=True, vertical_spacing=0.02)
for i, nw in enumerate(df_no_shift.dropna(axis=0).network.unique()):
    df_nw_hemi = df_no_shift[(df_no_shift.network == nw) & (df_no_shift.hemisphere == "LH")]
    fig.add_trace(go.Violin(x=df_nw_hemi.run, y=df_nw_hemi.correlation, name=f'LH {nw}', box_visible=True, meanline_visible=True, side="negative", legendgroup=f"LH {nw}", line_color=colors[i]), row=i%4+1, col=i//4+1)
    df_nw_hemi = df_no_shift[(df_no_shift.network == nw) & (df_no_shift.hemisphere == "RH")]
    fig.add_trace(go.Violin(x=df_nw_hemi.run, y=df_nw_hemi.correlation, name=f'RH {nw}', box_visible=True, meanline_visible=True, side="positive", legendgroup=f"RH {nw}", line_color=colors[i]), row=i%4+1, col=i//4+1)

fig.update_layout(title=f'Correlation between {feature_name} and parcels, {feature}')
fig.update_layout(width=900, height=800)
fig.show()

In [38]:
list(df_no_shift.network.unique()).remove(None)

In [ ]:
# !jupyter nbconvert --execute --to html correlation_statistics.ipynb